# Atelier de modélisation du métabolisme

### Maxime Mahout

## 1) Qu'est-ce que le métabolisme ?

L'ensemble des réactions qui se déroulent au sein d'une cellule.

![img](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6e/Metabolic_Metro_Map.svg/800px-Metabolic_Metro_Map.svg.png)

### -> Réseaux métaboliques:
* noeuds: métabolites
* arêtes: réaction
* pondération: stoechiométrie

\begin{equation}
\text{ex: } 2~ADP \rightarrow ATP + AMP
\end{equation}

## 2) Qu'est-ce qu'un modèle du métabolisme ?

Exemple: Mahout, Schwartz, Attal, Bakkar, Peres, 2024:

https://doi.org/10.1371/journal.pone.0313962

![img](https://journals.plos.org/plosone/article/figure/image?size=large&id=10.1371/journal.pone.0313962.g007)

Un modèle en sciences est une réduction d'un objet réel en une maquette que l'on peut analyser pour faire des prédictions que l'on espère avérées sur cet objet réel.

Ici, par exemple nous faisons des hypothèses sur la formation du stroma tumoral à partir d'un modèle réduit du métabolisme principal de la cellule cancéreuse.

## 3) Pourquoi modéliser le métabolisme ?

Comme en bio-statistique, on utilise les modèles afin de répondre à des questions scientifiques pour lesquelles on a formulé des hypothèses.

Egalement nous pouvons partir dépourvus d'une hypothèse et tester des propriétés sur des modèles pré-existants en espérant faire émerger des découvertes.


## 4) Comment modéliser le métabolisme ?

Il existe plusieurs façons de modéliser le métabolisme et sa complexité. Nous pouvons utiliser des approches 1) dynamiques ou 2) à l'état stationnaire pour modéliser les réactions biochimiques.

Les méthodes à l'état stationnaire, c'est à dire où la concentration en métabolite ne varie pas au cours du temps, telles que Flux Balance Analysis, sont les plus faciles d'accès.

Plus globalement afin d'obtenir des prédictions précises les modèles doivent respecter les principes de la "biologie des systèmes", une science qui s'attèle à représenter le système biologique dans toute sa complexité.

Autrement dit, le modèle biologique doit pouvoir intégrer le système "cellule" à travers tous ses niveaux: gène, ADN, ARN, molécules, chimie, protéine, enzyme, lipides, etc. et pourra être confirmé par tout autant d'analyses, génomique, transcriptomique, protéomique, métabolomique, etc. cf. https://www.nature.com/articles/nrm1857

## 5) Avec quelle méthode commencer ?

Je préconise la méthode "[Flux Balance Analysis](https://doi.org/10.1038/nbt.1614)" (FBA) et de partir "hypothesis-free" sur des modèles "à l'échelle du génome" préexistants, tels que ceux présents sur la base de données http://bigg.ucsd.edu/.

Afin de tester cela, nous pouvons installer l'outil `cobrapy` et tester le modèle par défaut `e_coli_core` (cf. [reference](https://europepmc.org/article/med/26443778)). Ainsi que `escher` pour la visualisation.

In [ ]:
pip install cobra

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install escher

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.1/124.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 2.5 MB/s eta 0:00:0000:0100:01mm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.1 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.

In [ ]:
!jupyter nbextension list

Known nbextensions:
  config dir: /root/.jupyter/nbconfig
    notebook section
      escher/extension  enabled 
      - Validating: OK
  config dir: /usr/local/etc/jupyter/nbconfig
    notebook section
      bqplot/extension  enabled 
      - Validating: OK
      catboost-widget/extension  enabled 
      - Validating: OK
      escher/extension  enabled 
      - Validating: OK
      ipyevents/extension  enabled 
      - Validating: problems found:
        - require?  X ipyevents/extension
      ipytree/extension  enabled 
      - Validating: OK
      jupyter-leaflet/extension  enabled 
      - Validating: OK
      jupyter-matplotlib/extension  enabled 
      - Validating: OK
      jupyterlab-plotly/extension  enabled 
      - Validating: OK
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
    tree section
      ipyparallel/main  enabled 
      - Validating: OK


Remarquer ici que l'extension escher/extension pour les Jupyter notebooks est activée.

In [ ]:
!jupyter nbextension enable --py escher

Enabling notebook extension escher/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
!jupyter labextension install escher

(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
Building jupyterlab assets (production, minimized)
An error occurred.
RuntimeError: JupyterLab failed to build
See the log file for details:  /tmp/jupyterlab-debug-s2u6t3qh.log


## https://github.com/jfoust-zymergen/escher/tree/python-3105) Visualisons un modèle métabolique

In [ ]:
import escher
from escher import Builder
import cobra

In [ ]:
builder = Builder()

In [ ]:
builder

Builder()

In [ ]:
!wget -nc http://bigg.ucsd.edu/static/models/e_coli_core.json

File ‘e_coli_core.json’ already there; not retrieving.



In [ ]:
builder = Builder(
    model_json='iAB_RBC_283.json'
)

In [ ]:
print(escher)

<module 'escher' from '/usr/local/lib/python3.10/dist-packages/escher/__init__.py'>


In [1]:
!python --version

Python 3.11.11


In [ ]:
builder

Builder()